In [15]:
import pandas as pd
import json
import utils

with open('data.json', 'r') as f:
    data = json.load(f)
    f.close()
alerts = pd.DataFrame(data['alerts'])

In [16]:
alerts = utils.separate_coords(alerts)

alerts.head(3)

,country,city,reportRating,reportByMunicipalityUser,confidence,reliability,type,uuid,roadType,magvar,subtype,street,pubMillis,endreport,x,y,geometry
0,CI,Antofagasta,1,false,5,10,HAZARD,a123f22e-e5e0-4c6c-8a4e-7434c4fd2110,6,17,HAZARD_ON_ROAD_CONSTRUCTION,Av. República de Croacia,1724417637000,1.724425e+12,-70.412429,-23.676846,POINT (-7838275.742 -2714079.538)
1,CI,Antofagasta,0,false,0,5,JAM,1d8df954-e380-4327-8015-f4fbadf74f2d,6,4,JAM_HEAVY_TRAFFIC,Av. Edmundo Pérez Zujovic,1724423878000,1.724425e+12,-70.392283,-23.617475,POINT (-7836033.099 -2706864.568)
2,CI,Antofagasta,0,false,0,5,JAM,c53a5bc6-f6b6-44b8-a5bd-af69b60a4580,1,359,,Pasaje El Volcán,1724425406000,1.724425e+12,-70.385619,-23.573323,POINT (-7835291.266 -2701501.184)


In [18]:
alerts = alerts[['pubMillis', 'endreport', 'x', 'y', 'geometry']]
alerts.head(3)

,pubMillis,endreport,x,y,geometry
0,1724417637000,1.724425e+12,-70.412429,-23.676846,POINT (-7838275.742 -2714079.538)
1,1724423878000,1.724425e+12,-70.392283,-23.617475,POINT (-7836033.099 -2706864.568)
2,1724425406000,1.724425e+12,-70.385619,-23.573323,POINT (-7835291.266 -2701501.184)


In [23]:
# Verifica si hay alertas cercanas en el mismo periodo de tiempo, con el fin de evaluar si hay reportes duplicados

for i in range(0, len(alerts)):
    q = 0
    for j in range(i+1, len(alerts)):
        dist = utils.haversine([alerts.iloc[i].x, alerts.iloc[j].x], [alerts.iloc[i].y, alerts.iloc[j].y])
        if dist < 300 and (alerts.iloc[j].pubMillis <= alerts.iloc[i].pubMillis <= alerts.iloc[j].endreport):
            q += 1
    alerts.loc[i, 'reports'] = q

alerts.head(3)


,pubMillis,endreport,x,y,geometry,reports
0,1724417637000,1.724425e+12,-70.412429,-23.676846,POINT (-7838275.742 -2714079.538),0.0
1,1724423878000,1.724425e+12,-70.392283,-23.617475,POINT (-7836033.099 -2706864.568),0.0
2,1724425406000,1.724425e+12,-70.385619,-23.573323,POINT (-7835291.266 -2701501.184),0.0


In [30]:
print('Hay {} alertas que se superponen'.format(alerts[alerts.reports > 0].shape[0]))
print('Por lo que se puede considerar como independientes unas de otras')

Hay 4 alertas que se superponen
Por lo que se puede considerar como independientes unas de otras
